In [ ]:
import pandas as pd

In [ ]:
df_trans = pd.read_csv('/kaggle/input/elo-merchant-category-recommendation/historical_transactions.csv', 
                      dtype={
                          'authorized_flag': 'string',
                          'card_id': 'string',
                          'category_1': 'string',
                          'category_3': 'string',
                          'merchant_id': 'string',
                          'purchase_date': 'string'
                      })
df_trans.info()

In [ ]:
df_trans

In [ ]:
df_merch = pd.read_csv('/kaggle/input/elo-merchant-category-recommendation/merchants.csv')

In [ ]:
df_merch

In [ ]:
df_merch_trans = pd.read_csv('/kaggle/input/elo-merchant-category-recommendation/new_merchant_transactions.csv')

In [ ]:
df_merch_trans

In [ ]:
df_train = pd.read_csv('/kaggle/input/elo-merchant-category-recommendation/train.csv')

In [ ]:
df_train

In [ ]:
df_test = pd.read_csv('/kaggle/input/elo-merchant-category-recommendation/test.csv')

In [ ]:
df_test

In [ ]:
import torch
x = torch.tensor(df_train[['feature_1', 'feature_2', 'feature_3']].to_numpy())
y = torch.tensor(df_train['target'].to_numpy()).unsqueeze(1)

In [ ]:
x = x/x.max()
x

In [ ]:
y = y/y.max()
y

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.25, random_state=1)

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.bn1 = torch.nn.BatchNorm1d(3)
        self.fc1 = torch.nn.Linear(3, 500)
        self.act1 = torch.nn.LeakyReLU()
        self.fc2 = torch.nn.Linear(500, 1)
    def forward(self, x):
        return self.fc2(self.act1(self.fc1(self.bn1(x))))

In [ ]:
import numpy as np
import copy
def train(model, optimizer, loss, x_train, y_train, x_val, y_val, n_epochs=30):
    device = ('cuda:0' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    train_loss_history = []
    val_loss_history = []
    best_params = None
    batch_size = 1000
    for epoch in range(1, n_epochs+1):
        model.train()
        order_train = np.random.permutation(x_train.size(0))
        running_loss_train = 0
        iters = 0
        for start_ind in range(0, x_train.size(0), batch_size):
            iters += 1
            indexes = order_train[start_ind:start_ind+batch_size]
            x_train_batch = x_train[indexes].to(device)
            y_train_batch = y_train[indexes].to(device)
            optimizer.zero_grad()
            y_train_batch_pred = model.forward(x_train_batch)
            loss_value_train = loss(y_train_batch_pred, y_train_batch)
            running_loss_train += loss_value_train.data.cpu()
            loss_value_train.backward()
            optimizer.step()
        train_loss = running_loss_train/iters
        train_loss_history.append(train_loss)
        
        model.eval()
        y_val_preds = net.forward(x_val.to(device))
        val_loss = loss(y_val_preds.to(device), y_val.to(device))
        val_loss_history.append(val_loss.data.cpu())
        if val_loss <= min(val_loss_history):
            best_params = copy.deepcopy(model.parameters)
        #print(f'epoch: {epoch:4d}; train loss: {train_loss:.4f}; val loss: {val_loss:.4f}')
    return train_loss_history, val_loss_history, best_params

In [ ]:
net = Net()
loss = torch.nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-4)

In [ ]:
x_train = x_train.float()
x_val = x_val.float()
y_train = y_train.float()
y_val = y_val.float()

In [ ]:
train_loss, val_loss, params = train(net, optimizer, loss, x_train, y_train, x_val, y_val, n_epochs = 100)

In [ ]:
net.parameters = copy.deepcopy(params)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(train_loss, c='b', label='train')
plt.plot(val_loss, c='r', label='val')
plt.grid()
plt.legend()
plt.title('loss history')

In [ ]:
loss(net.forward(x_val.to('cuda:0')), y_val.to('cuda:0'))

In [ ]:
df_test

In [ ]:
x_test = torch.tensor(df_test[['feature_1', 'feature_2', 'feature_3']].to_numpy()).float()

In [ ]:
y_test = net.forward(x_test.to('cuda:0'))

In [ ]:
answer = pd.DataFrame(df_test['card_id'])
answer['target'] = y_test.data.cpu()

In [ ]:
answer

In [ ]:
answer.to_csv('/kaggle/working/submissions.csv', index=False)

In [ ]:
df = pd.read_csv('/kaggle/working/submissions.csv')

In [ ]:
df

In [ ]:
answer